<a href="https://colab.research.google.com/github/naphatsiri/M.Sc-IS-Fault-Detection/blob/main/IS_fault_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import general libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#import model libraries


## 1.Data-preprocessing

In [ ]:
path = '/content/normal cf.xls'

df = pd.read_excel(path)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5191 entries, 0 to 5190
Data columns (total 66 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Time                 5191 non-null   float64
 1   TWE_set              5191 non-null   int64  
 2   TEI                  5191 non-null   float64
 3   TWEI                 5191 non-null   float64
 4   TEO                  5191 non-null   float64
 5   TWEO                 5191 non-null   float64
 6   TCI                  5191 non-null   float64
 7   TWCI                 5191 non-null   float64
 8   TCO                  5191 non-null   float64
 9   TWCO                 5191 non-null   float64
 10  TSI                  5191 non-null   float64
 11  TSO                  5191 non-null   float64
 12  TBI                  5191 non-null   float64
 13  TBO                  5191 non-null   float64
 14  Cond Tons            5191 non-null   float64
 15  Cooling Tons         5191 non-null   f

In [ ]:
df.head(10)

,Time,TWE_set,TEI,TWEI,TEO,TWEO,TCI,TWCI,TCO,TWCO,...,VC,VE,VW,TWI,TWO,THI,THO,FWW,FWH,FWB
0,3.578,50,70.88,70.9,71.35,71.2,71.33,71.2,71.98,71.8,...,57.0,56,0.0,66.21,70.65,70.28,70.84,7.738000e-46,2.251000e-45,2.591000e-45
1,13.570,50,70.88,70.9,71.35,71.2,71.33,71.2,72.02,71.8,...,57.0,56,30.0,66.21,70.65,70.24,70.84,8.095000e-46,2.251000e-45,2.591000e-45
2,23.560,50,70.88,71.2,71.39,71.4,71.37,71.5,71.98,71.4,...,57.0,56,30.0,66.21,70.65,70.20,70.80,3.404000e+01,9.281000e+01,1.128000e+02
3,33.570,50,70.92,71.2,71.43,71.4,71.37,70.9,71.94,71.2,...,57.0,56,30.0,66.21,70.52,70.94,70.76,5.159000e+01,1.152000e+02,1.568000e+02
4,43.580,50,70.92,71.2,71.43,71.2,71.37,71.4,71.81,71.2,...,57.0,56,30.0,65.92,70.40,70.90,70.84,4.292000e+01,1.116000e+02,1.478000e+02
5,53.570,50,71.00,71.4,71.35,71.1,71.33,70.9,71.69,71.2,...,57.0,56,30.0,65.23,70.19,71.10,70.96,3.038000e+01,8.547000e+01,1.624000e+02
6,63.570,50,71.08,71.4,71.35,71.2,71.28,70.8,71.65,70.9,...,57.0,56,30.0,64.37,69.74,71.18,71.09,2.806000e+01,8.586000e+01,1.795000e+02
7,73.570,50,71.12,71.2,71.35,71.2,71.20,70.8,71.53,70.9,...,57.0,56,30.0,63.80,69.29,71.23,71.13,2.532000e+01,6.816000e+01,1.961000e+02
8,83.570,50,71.16,71.5,71.35,71.1,71.20,70.8,71.41,70.8,...,57.0,56,30.0,63.30,68.68,71.27,71.17,2.587000e+01,5.933000e+01,2.046000e+02
9,93.580,50,71.16,71.2,71.35,71.2,71.08,70.4,71.28,70.6,...,57.0,56,30.0,63.02,68.23,71.35,71.21,2.469000e+01,4.204000e+01,2.143000e+02


## 2.Feature-engineering